<a href="https://colab.research.google.com/github/arijit1995/SPH_Py/blob/master/SPH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library import

In [ ]:
from numba import jit,prange,vectorize,float64,unicode
from numba.typed import Dict
from numba.types import unicode_type
import numpy as np

# Kernel Definition

In [ ]:
@vectorize([float64(float64,float64)])
def quartic_spline_kernel(x,h):
    q=x/h
    alpha=15/(7*np.pi*h*h)
    if q>2:
        return 0
    else:
        return ((2/3)-(9*q*q/8)+(19*q*q*q/24)-5*(q**4)/24)*alpha

In [ ]:
@vectorize([float64(float64,float64)])
def quartic_kernel_derivative(x,h):
    q=x/h
    alpha=15/(7*np.pi*h*h)
    if q<=2:
        return alpha*((57*(q**2)/24)-(18*q/8)-(20*(q**3)/32))
    else:
        return 0

# Particle Array Generation

In [ ]:
def get_particle_array(x,y,h,*properties):
    material=Dict.empty(key_type=unicode_type,value_type=float64[:])
    material["x"]=x
    material["y"]=y
    material["h"]=h
    for i in properties:
        material[i]=np.zeros_like(x)
    return material

In [ ]:
def get_rigid_particle_array(x,y,h,*properties,cg_x,u_x,mass):
    material=Dict.empty(key_type=unicode_type,value_type=float64[:])
    material["x"]=x
    material["y"]=y
    material["h"]=h
    for i in properties:
        material[i]=np.zeros_like(x)
    material["x_cg"]=cg_x
    material["u_cg"]=cg_u
    material["Total_mass"]=mass

# Equations

In [ ]:
@jit(nopython=True,fastmath=True,parallel=True)
def mass_conservation(domain,source):
    temp=np.zeros_like(domain["x"])
    for i in prange(len(temp)):
        dist=((source["x"]-domain["x"][i])**2+(source["y"]-domain["y"][i])**2)**0.5
        deriv=quartic_kernel_derivative(dist,domain["h"][i])[dist*(dist-3*domain["h"][i])<0]
        dot_prod=(source["u"][dist*(dist-3*domain["h"][i])<0]-domain["u"][i])*(source["x"][dist*(dist-3*domain["h"][i])<0]-domain["x"][i])+(source["v"][dist*(dist-3*domain["h"][i])<0]-domain["v"][i])*(source["y"][dist*(dist-3*domain["h"][i])<0]-domain["y"][i])
        temp[i]=np.sum(source["m"][dist*(dist-3*domain["h"][i])<0]*dot_prod*deriv/dist[dist*(dist-3*domain["h"][i])<0])/domain["h"][i]
    domain["drho/dt"]+=temp

In [ ]:
@jit(nopython=True,fastmath=True,parallel=True)
def momentum_conservation_pressure(domain,source):
    temp_u=np.zeros_like(domain["x"])
    temp_v=np.zeros_like(domain["y"])
    for i in prange(len(temp_u)):
        dist=((source["x"]-domain["x"][i])**2+(source["y"]-domain["y"][i])**2)**0.5
        deriv=quartic_kernel_derivative(dist,domain["h"][i])[dist*(dist-3*domain["h"][i])<0]
        pressure_term=(source["P"][dist*(dist-3*domain["h"][i])<0]/(source["rho"][dist*(dist-3*domain["h"][i])<0]**2)+(domain["P"][i]/(domain["rho"][i]**2)))*deriv*source["m"][dist*(dist-3*domain["h"][i])<0]/dist[dist*(dist-3*domain["h"][i])<0]
        temp_u[i]=np.sum(pressure_term*(source["x"][dist*(dist-3*domain["h"][i])<0]-domain["x"][i]))/domain["h"][i]
        temp_v[i]=np.sum(pressure_term*(source["y"][dist*(dist-3*domain["h"][i])<0]-domain["y"][i]))/domain["h"][i]
    domain["du/dt"]+=temp_u
    domain["dv/dt"]+=temp_v        

In [ ]:
@jit(nopython=True,fastmath=True,parallel=True)
def momentum_conservation_viscosity(domain,source,nu):
    temp_u=np.zeros_like(domain["x"])
    temp_v=np.zeros_like(domain["y"])
    for i in prange(len(temp_u)):
        dist=((source["x"]-domain["x"][i])**2+(source["y"]-domain["y"][i])**2)**0.5
        deriv=quartic_kernel_derivative(dist,domain["h"][i])[dist*(dist-3*domain["h"][i])<0]
        term=(2*nu*source["m"][dist*(dist-3*domain["h"][i])<0]/source["rho"][dist*(dist-3*domain["h"][i])<0])*deriv/dist[dist*(dist-3*domain["h"][i])<0]
        temp_u[i]=np.sum(term*(source["u"][dist*(dist-3*domain["h"][i])<0]-domain["u"][i]))/domain["h"][i]
        temp_v[i]=np.sum(term*(source["v"][dist*(dist-3*domain["h"][i])<0]-domain["v"][i]))/domain["h"][i]
    domain["du/dt"]+=temp_u
    domain["dv/dt"]+=temp_v

In [ ]:
@jit(nopython=True,fastmath=True,parallel=True)
def energy_conservation(domain,source):                       # Energy Equation contribution of Pressure
    temp_e=np.zeros_like(domain["x"])
    for i in prange(len(temp_e)):
        dist=((source["x"]-domain["x"][i])**2+(source["y"]-domain["y"][i])**2)**0.5
        deriv=quartic_kernel_derivative(dist,domain["h"][i])[dist*(dist-3*domain["h"][i])<0]
        dot_prod=(source["u"][dist*(dist-3*domain["h"][i])<0]-domain["u"][i])*(source["x"][dist*(dist-3*domain["h"][i])<0]-domain["x"][i])+(source["v"][dist*(dist-3*domain["h"][i])<0]-domain["v"][i])*(source["y"][dist*(dist-3*domain["h"][i])<0]-domain["y"][i])
        temp_e[i]=np.sum(source["m"][dist*(dist-3*domain["h"][i])<0]*dot_prod*deriv*source["P"][dist*(dist-3*domain["h"][i])<0]/dist[dist*(dist-3*domain["h"][i])<0])/domain["h"][i]
    domain["de/dt"]-=temp_e
         

In [ ]:
@jit(nopython=True,fastmath=True,parallel=True)
def XSPH_correction(domain,source,timestep,eps=0.5):    # XSPH Correction for accurate prediction of Velocities
    temp_u=np.zeros_like(domain["x"])
    temp_v=np.zeros_like(domain["x"])
    for i in prange(len(temp_u)):
        dist=((source["x"]-domain["x"][i])**2+(source["y"]-domain["y"][i])**2)**0.5
        W=quartic_spline_kernel(dist,domain["h"][i])
        term=2*W*eps*source["m"]/(source["rho"]+domain["rho"][i])
        temp_u[i]=np.sum(term*(source["u"]-domain["u"][i]))
        temp_v[i]=np.sum(term*(source["v"]-domain["v"][i]))
    domain["x"]+=temp_u*timestep
    domain["y"]+=temp_v*timestep

In [ ]:
@jit(nopython=True,fastmath=True,parallel=True)
def SPH_strain_rates(domain,source):       # Compute Strain Rate Components
    temp_u_x=np.zeros_like(domain["x"])
    temp_u_y=np.zeros_like(domain["x"])
    temp_v_x=np.zeros_like(domain["x"])
    temp_v_y=np.zeros_like(domain["x"])
    for i in prange(len(temp_u_x)):
        dist=((source["x"]-domain["x"][i])**2+(source["y"]-domain["y"][i])**2)**0.5
        deriv=quartic_kernel_derivative(dist,domain["h"][i])[dist*(dist-3*domain["h"][i])<0]
        value=(source["m"][dist*(dist-3*domain["h"][i])<0]/source["rho"][dist*(dist-3*domain["h"][i])<0])*deriv[dist*(dist-3*domain["h"][i])<0]/dist[dist*(dist-3*domain["h"][i])<0]
        temp_u_x[i]=np.sum(value*(source["u"][dist*(dist-3*domain["h"][i])<0]-domain["u"][i])*(source["x"][dist*(dist-3*domain["h"][i])<0]-domain["x"][i]))/domain["h"][i]
        temp_u_y[i]=np.sum(value*(source["u"][dist*(dist-3*domain["h"][i])<0]-domain["u"][i])*(source["y"][dist*(dist-3*domain["h"][i])<0]-domain["y"][i]))/domain["h"][i]
        temp_v_x[i]=np.sum(value*(source["v"][dist*(dist-3*domain["h"][i])<0]-domain["v"][i])*(source["x"][dist*(dist-3*domain["h"][i])<0]-domain["x"][i]))/domain["h"][i]
        temp_v_y[i]=np.sum(value*(source["v"][dist*(dist-3*domain["h"][i])<0]-domain["v"][i])*(source["y"][dist*(dist-3*domain["h"][i])<0]-domain["y"][i]))/domain["h"][i]
    domain["du/dx"]+=temp_u_x
    domain["du/dy"]+=temp_u_y
    domain["dv/dx"]+=temp_v_x
    domain["dv/dy"]+=temp_v_y
       

In [ ]:
@jit(nopython=True,fastmath=True,parallel=True)
def Hooke_devaiatoric_stress_rate(domain):
    #Defining Strains and Rotations From Displacements
    
    e_xx=domain["du/dx"]
    e_yy=domain["dv/dy"]
    e_xy=e_yx=0.5*(domain["du/dy"]+domain["dv/dx"])
    omega_xx=np.zeros_like(e_xx)
    omega_yy=np.zeros_like(e_yy)
    omega_xy=0.5*(domain["du/dy"]-domain["dv/dx"])
    omega_yx=(-1)*omega_xy
    
    # Main Equations
    
    domain["dsxx/dt"]=(4*domain["G"]/3)*e_xx+2*domain["sxx"]*omega_xx+omega_xy*(domain["sxy"]+domain["syx"])
    domain["dsxy/dt"]=(2*domain["G"])*e_xy+domain["sxx"]*omega_yx+domain["sxy"]*omega_yy+domain["sxy"]*omega_xx+domain["syy"]*omega_xy
    domain["dsyx/dt"]=(2*domain["G"])*e_yx+domain["syy"]*omega_xy+domain["syx"]*omega_xx+domain["syx"]*omega_yy+domain["sxx"]*omega_yx
    domain["dsyy/dt"]=(4*domain["G"]/3)*e_yy+2*domain["syy"]*omega_yy+omega_yx*(domain["syx"]+domain["sxy"])
    

In [ ]:
@jit(nopython=True,fastmath=True,parallel=True)
def Momentum_Equation_deviatoric_stress_term(domain,source):
    temp_u=np.zeros_like(domain["x"])
    temp_v=np.zeros_like(domain["y"])
    for i in prange(len(temp_u)):
        dist=((source["x"]-domain["x"][i])**2+(source["y"]-domain["y"][i])**2)**0.5
        deriv=quartic_kernel_derivative(dist,domain["h"][i])[dist*(dist-3*domain["h"][i])<0]
        pressure_term1=(source["sxx"][dist*(dist-3*domain["h"][i])<0]/(source["rho"][dist*(dist-3*domain["h"][i])<0]**2)+domain["sxx"][i]/(domain["rho"][i]**2))
        pressure_term2=(source["sxy"][dist*(dist-3*domain["h"][i])<0]/(source["rho"][dist*(dist-3*domain["h"][i])<0]**2)+domain["sxy"][i]/(domain["rho"][i]**2))
        pressure_term3=(source["syx"][dist*(dist-3*domain["h"][i])<0]/(source["rho"][dist*(dist-3*domain["h"][i])<0]**2)+domain["syx"][i]/(domain["rho"][i]**2))
        pressure_term4=(source["syy"][dist*(dist-3*domain["h"][i])<0]/(source["rho"][dist*(dist-3*domain["h"][i])<0]**2)+domain["syy"][i]/(domain["rho"][i]**2))
        mass_term=deriv*source["m"][dist*(dist-3*domain["h"][i])<0]/dist[dist*(dist-3*domain["h"][i])<0]
        temp_u[i]=np.sum(mass_term*((pressure_term1*(source["x"][dist*(dist-3*domain["h"][i])<0]-domain["x"][i]))+(pressure_term2*(source["y"][dist*(dist-3*domain["h"][i])<0]-domain["y"][i]))))/domain["h"][i]
        temp_v[i]=np.sum(mass_term*((pressure_term3*(source["x"][dist*(dist-3*domain["h"][i])<0]-domain["x"][i]))+(pressure_term4*(source["y"][dist*(dist-3*domain["h"][i])<0]-domain["y"][i]))))/domain["h"][i]
    domain["du/dt"]+=temp_u
    domain["dv/dt"]+=temp_v



In [ ]:
 @jit(nopython=True,fastmath=True,parallel=True)
 def Energy_Equation_deviatoric_stress_term(domain):
   d_eps_xx=domain["du/dx"]
   d_eps_xy=0.5*(domain["du/dy"]+domain["dv/dx"])
   d_eps_yx=0.5*(domain["du/dy"]+domain["dv/dx"])
   d_eps_yy=domain["dv/dy"]
   temp_e=(domain["dsxx/dt"]*d_eps_xx)+(domain["dsxy/dt"]*d_eps_xy)+(domain["dsyx/dt"]*d_eps_yx)+(domain["dsyy/dt"]*d_eps_yy)
   domain["de/dt"]+=temp_e

In [ ]:
@jit(nopython=True,fastmath=True,parallel=True)
def Euler_time_evolve(domain,timestep):                         # Euler time evolve for General  Particle
    for i in domain.keys():
        if "/dt" in i:
            domain[i[1:len(i)-3]]+=domain[i]*timestep
            domain[i]=np.zeros_like(domain[i])
    domain["x"]+=domain["u"]*timestep
    domain["y"]+=domain["v"]*timestep

In [ ]:
@vectorize([float64(float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64)])
def Artificial_viscosity_value(x1,y1,x2,y2,c1,c2,rho1,rho2,h,u1,u2,v1,v2,alpha,beta): # Compute Artificial Viscosity
    dot_prod=(u1-u2)*(x1-x2)+(v1-v2)*(y1-y2)
    if dot_prod>=0:
        return 0
    else:
        dist=((x2-x1)**2+(y2-y1)**2)**0.5
        c=0.5*(c1+c2)
        rho=0.5*(rho1+rho2)
        chi=(h*dot_prod)/(0.01*h*h+dist**2)
        val=(beta*(chi**2)-alpha*c*chi)/rho
        return val

In [ ]:
@jit(nopython=True,fastmath=True,parallel=True)
def artificial_viscosity_momentum_equation_term(source,domain,alpha,beta):  # Shock Fitting in Momentum Equation
    temp_u=np.zeros_like(domain["x"])
    temp_v=np.zeros_like(domain["y"])
    for i in prange(len(temp_u)):
        dist=((source["x"]-domain["x"][i])**2+(source["y"]-domain["y"][i])**2)**0.5
        deriv=quartic_kernel_derivative(dist,domain["h"][i])[dist*(dist-3*domain["h"][i])<0]
        temp_visc=Artificial_viscosity_value(source["x"][dist*(dist-3*domain["h"][i])<0],source["y"][dist*(dist-3*domain["h"][i])<0],domain["x"][i],domain["y"][i],source["c"][dist*(dist-3*domain["h"][i])<0],domain["c"][i],source["rho"][dist*(dist-3*domain["h"][i])<0],domain["rho"][i],domain["h"][i],source["u"][dist*(dist-3*domain["h"][i])<0],domain["u"][i],source["v"][dist*(dist-3*domain["h"][i])<0],source["v"][i],alpha,beta)
        temp_u[i]=np.sum(source["m"][dist*(dist-3*domain["h"][i])<0]*temp_visc*deriv*(source["x"][dist*(dist-3*domain["h"][i])<0]-domain["x"][i]))/domain["h"][i]
        temp_v[i]=np.sum(source["m"][dist*(dist-3*domain["h"][i])<0]*temp_visc*deriv*(source["y"][dist*(dist-3*domain["h"][i])<0]-domain["y"][i]))/domain["h"][i]
    domain["du/dt"]+=temp_u
    domain["dv/dt"]+=temp_v  
        
        

In [ ]:
@jit(nopython=True,fastmath=True,parallel=True)
def artificial_viscosity_energy_equation_term(source,domain,alpha,beta): #Capture Shock in Energy Equation
    temp_e=np.zeros_like(domain["x"])
    for i in prange(len(temp_e)):
        dist=((source["x"]-domain["x"][i])**2+(source["y"]-domain["y"][i])**2)**0.5
        deriv=quartic_kernel_derivative(dist,domain["h"][i])[dist*(dist-3*domain["h"][i])<0]
        dot_prod=(source["u"][dist*(dist-3*domain["h"][i])<0]-domain["u"][i])*(source["x"][dist*(dist-3*domain["h"][i])<0]-domain["x"][i])+(source["v"][dist*(dist-3*domain["h"][i])<0]-domain["v"][i])*(source["y"][dist*(dist-3*domain["h"][i])<0]-domain["y"][i])
        temp_visc=Artificial_viscosity_value(source["x"][dist*(dist-3*domain["h"][i])<0],source["y"][dist*(dist-3*domain["h"][i])<0],domain["x"][i],domain["y"][i],source["c"][dist*(dist-3*domain["h"][i])<0],domain["c"][i],source["rho"][dist*(dist-3*domain["h"][i])<0],domain["rho"][i],domain["h"][i],source["u"][dist*(dist-3*domain["h"][i])<0],domain["u"][i],source["v"][dist*(dist-3*domain["h"][i])<0],source["v"][i],alpha,beta)
        temp_e[i]=np.sum(dot_prod*deriv*temp_visc*source["m"][dist*(dist-3*domain["h"][i])<0])/domain["h"][i]
    domain["de/dt"]+=temp_e
        

In [ ]:
@vectorize([float64(float64)])
def RELU(x):
    if x>0:
        return -1
    else:
        return 0

In [ ]:
@jit(nopython=True,fastmath=True,parallel=True)
def compute_average_distance(x_arr,y_arr):
    n=len(x_arr)
    sum_points=0.5*n*(n-1)
    su=0
    for i in prange(0,len(x_arr)):
        dist=((x_arr-x_arr[i])**2+(y_arr-y_arr[i])**2)**0.5
        su+=np.sum(dist)
    return su/sum_points

In [ ]:
@jit(nopython=True,fastmath=True)
def JWL_EOS(domain,A,B,R1,R2,omega,Rho0):
  V=Rho0/domain["rho"]
  domain["P"]=A*(1-(omega/(R1*V)))*np.exp(-1*R1*V)+B*(1-(omega/(R2*V)))*np.exp(-1*R2*V)+((omega*domain["e"])/V)


In [ ]:
@jit(nopython=True,fastmath=True)
def update_smoothing_length(domain,h0,rho0,dim=2):
  domain["h"]=h0*((rho0/domain["rho"])**(1/dim))

In [ ]:
@jit(nopython=True,fastmath=True)
def Isothermal_EOS(domain,rho0,c0,P0):
  domain["P"]=P0+(c0**2)*(rho0-domain["rho"])

In [ ]:
@jit(nopython=True,fastmath=True)
def Stiffened_Gas_EOS(domain,gamma,c0,rho0): # Gruneisen Gamma ,Unpert sound speed, Initial Density
  domain["P"]=c0*c0*(domain["rho"]-rho0)+(gamma-1)*domain["rho"]*domain["e"]
  domain["c"]=(c0*c0+(gamma-1)*(domain["e"]+(domain["P"]/domain["rho"])))**0.5

In [ ]:
@jit(nopython=True,fastmath=True)
def Mie_Gruneisen_EOS(domain,gamma,rho0,c0,S): # Gruneisen Gamma ,Initial Density ,Unpert sound speed, S
  a=rho0*c0*c0
  b=a*(1+2*(S-1))
  c=a*(2*(S-1)+3*((S-1)**2))
  ratio=(domain["rho"]/rho0)-1
  ratio2=(ratio1**2)
  PH=a*ratio
  PH[ratio>0]=PH[ratio>0]+ratio2[ratio>0]*(b+c*ratio[ratio>0])
  domain["P"]=(1-0.5*gamma*ratio)*PH+domain["rho"]*domain["e"]*gamma
  

In [ ]:
@jit(nopython=True,fastmath=True)
def P_alpha_EOS_for_concrete(domain):
  

SyntaxError: ignored

In [ ]:
@jit(nopython=True,fastmath=True)
def CDP_Damage_model_for_concrete(domain):
  

In [ ]:
@jit(nopython=True,fastmath=True)
def Soil_damage_Plasticity_model(domain):
  

In [ ]:
@jit(nopython=True,fastmath=True,parallel=True)
def energy_conservation_efficient_pressure(domain,source):
  temp_e=np.zeros_like(domain["x"])
  for i in prange(len(temp_e)):
    dist=((source["x"]-domain["x"][i])**2+(source["y"]-domain["y"][i])**2)**0.5
    deriv=quartic_kernel_derivative(dist,domain["h"][i])[dist*(dist-3*domain["h"][i])<0]
    pressure_term=(source["P"][dist*(dist-3*domain["h"][i])<0]/(source["rho"][dist*(dist-3*domain["h"][i])<0]**2)+(domain["P"][i]/(domain["rho"][i]**2)))*deriv*source["m"][dist*(dist-3*domain["h"][i])<0]/dist[dist*(dist-3*domain["h"][i])<0]
    dot_prod=(source["u"][dist*(dist-3*domain["h"][i])<0]-domain["u"][i])*(source["x"][dist*(dist-3*domain["h"][i])<0]-domain["x"][i])+(source["v"][dist*(dist-3*domain["h"][i])<0]-domain["v"][i])*(source["y"][dist*(dist-3*domain["h"][i])<0]-domain["y"][i])
    temp_e[i]=np.sum(pressure_term*dot_prod)/domain["h"][i]
  domain["de/dt"]+=temp_e  

In [ ]:
@jit(nopython=True,fastmath=True)
def detonation_time_evolve(domain,point_x,point_y,Wavefront_Velocity,timestep,current_time):
  dist=((domain["x"]-point_x)**2+(domain["y"]-point_y)**2)**0.5
  points=(dist<=(Wavefront_Velocity*current_time))
  outside=(dist>=(Wavefront_Velocity*current_time))
  for key in domain.keys():
    if "/dt" in key:
      domain[key[1:(len(key)-3)]][points]=domain[key][points]*timestep
      domain[key]=np.zeros_like(domain[key])
  domain["x"]+=domain["u"]*timestep
  domain["y"]+=domain["v"]*timestep
  domain["P"][outside]=0

In [ ]:
@jit(nopython=True,fastmath=True)
def Body_Forces(domain,f_x,f_y):
  domain["du/dt"]+=f_x/domain["m"]
  domain["dv/dt"]+=f_y/domain["m"]


In [ ]:
@jit(nopython=True,fastmath=True)
def Dynamic_Boundary_Time_evolve(domain,timestep):
  for key in domain.keys():
    if "/dt" in key and "du" not in key and "dv" not in key:
      domain[key[1:len(key)-3]]=domain[key]*timestep

In [ ]:
@jit(nopython=True,fastmath=True,parallel=True)
def Monaghan_Artificial_Stress_momentum_equation(domain,source,epsilon=0.3,n=3):
  temp_u=np.zeros_like(domain["x"])
  temp_v=np.zeros_like(domain["y"])
  for i in prange(len(temp_u)):
    dist=((source["x"]-domain["x"][i])**2+(source["y"]-domain["y"][i])**2)**0.5
    deriv=quartic_kernel_derivative(dist,domain["h"][i])[dist*(dist-3*domain["h"][i])<0]
    sxx=source["sxx"][dist*(dist-3*domain["h"][i])<0]-source["P"][dist*(dist-3*domain["h"][i])<0]
    syy=source["syy"][dist*(dist-3*domain["h"][i])<0]-source["P"][dist*(dist-3*domain["h"][i])<0]
    sxy=source["sxy"][dist*(dist-3*domain["h"][i])<0]
    dxx=domain["sxx"][i]-domain["P"][i]
    dyy=domain["syy"][i]-domain["P"][i]
    dxy=domain["sxy"][i]
    rho_s=source["rho"][dist*(dist-3*domain["h"][i])<0]
    rho_d=domain["rho"][i]
    theta_s=0.5*np.arctan(2*sxy/(sxx-syy))
    theta_d=0.5*np.arctan(2*dxy/(dxx-dyy))
    sxx_bar=(np.cos(theta_s)**2)*sxx+(np.sin(theta_s)**2)*syy+(np.sin(2*theta_s))*sxy
    syy_bar=(np.sin(theta_s)**2)*sxx+(np.cos(theta_s)**2)*syy+(np.sin(2*theta_s))*sxy
    dxx_bar=(np.cos(theta_d)**2)*dxx+(np.sin(theta_d)**2)*dyy+(np.sin(2*theta_d))*dxy
    dyy_bar=(np.sin(theta_d)**2)*dxx+(np.cos(theta_d)**2)*dyy+(np.sin(2*theta_d))*dxy
    r_sxx_bar=epsilon*RELU(sxx_bar)*sxx_bar/(rho_s**2)
    r_syy_bar=epsilon*RELU(syy_bar)*syy_bar/(rho_s**2)
    r_dxx_bar=epsilon*RELU(dxx_bar)*dxx_bar/(rho_d**2)
    r_dyy_bar=epsilon*RELU(dyy_bar)*dyy_bar/(rho_d**2)
    r_sxx=(np.cos(theta_s)**2)*r_sxx_bar+(np.sin(theta_s)**2)*r_syy_bar
    r_syy=(np.sin(theta_s)**2)*r_sxx_bar+(np.cos(theta_s)**2)*r_syy_bar
    r_sxy=(0.5*np.sin(2*theta_s))*(r_sxx_bar-r_syy_bar)
    r_dxx=(np.cos(theta_d)**2)*r_dxx_bar+(np.sin(theta_d)**2)*r_dyy_bar
    r_dyy=(np.sin(theta_d)**2)*r_dxx_bar+(np.cos(theta_d)**2)*r_dyy_bar
    r_dxy=(0.5*np.sin(2*theta_d))*(r_dxx_bar-r_dyy_bar)
    f_avg=quartic_spline_kernel(compute_average_distance(source["x"][dist*(dist-3*domain["h"][i])<0],source["y"][dist*(dist-3*domain["h"][i])<0]),domain["h"][i])
    f_ab=((quartic_spline_kernel(dist[dist*(dist-3*domain["h"][i])<0],domain["h"][i]))/f_avg)**n
    k_xx=(r_sxx+r_dxx)*f_ab
    k_yy=(r_syy+r_dyy)*f_ab
    k_xy=(r_sxy+r_dxy)*f_ab
    temp_u[i]=np.sum((source["m"][dist*(dist-3*domain["h"][i])<0]*deriv/dist[dist*(dist-3*domain["h"][i])<0])*((source["x"][dist*(dist-3*domain["h"][i])<0]-domain["x"][i])*k_xx+(source["y"][dist*(dist-3*domain["h"][i])<0]-domain["y"][i])*k_xy))/domain["h"][i]
    temp_v[i]=np.sum((source["m"][dist*(dist-3*domain["h"][i])<0]*deriv/dist[dist*(dist-3*domain["h"][i])<0])*((source["x"][dist*(dist-3*domain["h"][i])<0]-domain["x"][i])*k_xy+(source["y"][dist*(dist-3*domain["h"][i])<0]-domain["y"][i])*k_yy))/domain["h"][i]
  domain["du/dt"]+=temp_u
  domain["dv/dt"]+=temp_v
    




In [ ]:
x=np.arange(-5e-2,5e-2,2.5e-3)
y=np.arange(-5e-2,5e-2,2.5e-3)
[xx,yy]=np.meshgrid(x,y)
mm=xx[(xx)**2+(yy)**2<=25e-4]
nn=yy[(xx)**2+(yy)**2<=25e-4]
mm=mm.ravel()
nn=nn.ravel()
Body=Dict()
Body["x"]=mm
Body["y"]=nn
Body["h"]=1.2*2.5e-3*np.ones_like(mm)
Body["rho"]=1630*np.ones_like(mm)
Body["drho/dt"]=np.zeros_like(mm)
Body["u"]=np.zeros_like(mm)
Body["v"]=np.zeros_like(mm)
Body["du/dt"]=np.zeros_like(mm)
Body["dv/dt"]=np.zeros_like(mm)
Body["P"]=np.zeros_like(mm)
Body["e"]=np.ones_like(mm)*(4.294e6)
Body["de/dt"]=np.zeros_like(mm)
Body["m"]=1630*2.5e-3*2.5e-3*np.ones_like(mm)
Body["sxx"]=np.zeros_like(mm)
Body["sxy"]=np.zeros_like(mm)
Body["syx"]=np.zeros_like(mm)
Body["syy"]=np.zeros_like(mm)
Body["dsxx/dt"]=np.zeros_like(mm)
Body["dsxy/dt"]=np.zeros_like(mm)
Body["dsyx/dt"]=np.zeros_like(mm)
Body["dsyy/dt"]=np.zeros_like(mm)
Body["du/dx"]=np.zeros_like(mm)
Body["du/dy"]=np.zeros_like(mm)
Body["dv/dx"]=np.zeros_like(mm)
Body["dv/dy"]=np.zeros_like(mm)
Body["G"]=1e7*np.ones_like(mm)

In [ ]:
%timeit Monaghan_Artificial_Stress_momentum_equation(Body,Body)

1247

In [ ]:
timestep=2.5e-10
current_time=0
iterations=100000
pos_x=[]
pos_y=[]
pos_e=[]
for i in range(iterations):
  mass_conservation(Body,Body)
  momentum_conservation_pressure(Body,Body)
  #energy_conservation(Body,Body)
  energy_conservation_efficient_pressure(Body,Body)
  JWL_EOS(Body,371.25e9,3.231e9,4.15,0.95,0.3,1630)
  update_smoothing_length(Body,1.2*2.5e-3,1630)
  detonation_time_evolve(Body,0,0,6930,timestep,current_time)
  #Euler_time_evolve(Body,timestep)
  current_time+=timestep
  pos_x.append(Body["x"]+0)
  pos_y.append(Body["y"]+0)
  pos_e.append(Body["e"]+0)
  print("Iteration "+str(i+1)+" Complete")

Streaming output truncated to the last 5000 lines.
Iteration 5322 Complete
Iteration 5323 Complete
Iteration 5324 Complete
Iteration 5325 Complete
Iteration 5326 Complete
Iteration 5327 Complete
Iteration 5328 Complete
Iteration 5329 Complete
Iteration 5330 Complete
Iteration 5331 Complete
Iteration 5332 Complete
Iteration 5333 Complete
Iteration 5334 Complete
Iteration 5335 Complete
Iteration 5336 Complete
Iteration 5337 Complete
Iteration 5338 Complete
Iteration 5339 Complete
Iteration 5340 Complete
Iteration 5341 Complete
Iteration 5342 Complete
Iteration 5343 Complete
Iteration 5344 Complete
Iteration 5345 Complete
Iteration 5346 Complete
Iteration 5347 Complete
Iteration 5348 Complete
Iteration 5349 Complete
Iteration 5350 Complete
Iteration 5351 Complete
Iteration 5352 Complete
Iteration 5353 Complete
Iteration 5354 Complete
Iteration 5355 Complete
Iteration 5356 Complete
Iteration 5357 Complete
Iteration 5358 Complete
Iteration 5359 Complete
Iteration 5360 Complete
Iteration 536

KeyboardInterrupt: ignored

In [ ]:
from ipywidgets import interact,IntSlider
import matplotlib.pyplot as plt

def printer(val):
  plt.scatter(pos_x[val],pos_y[val],c=pos_e[val])

interact(printer,val=IntSlider(min=0,max=len(pos_x)-1,step=100))

interactive(children=(IntSlider(value=0, description='val', max=10319, step=100), Output()), _dom_classes=('wi…

<function __main__.printer>

In [ ]:
mass_conservation(Body,Body)

In [ ]:
mass_conservation.parallel_diagnostics(level=1)

 
 Parallel Accelerator Optimizing:  Function mass_conservation, <ipython-
input-6-8881832c0a64> (1)  


Parallel loop listing for  Function mass_conservation, <ipython-input-6-8881832c0a64> (1) 
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|loop #ID
@jit(nopython=True,fastmath=True,parallel=True)                                                                                                                                                                                                                                          | 
def mass_conservation(domain,source):                                                                                                                                                                                                

In [ ]:
mass_conservation(Body,Body)
momentum_conservation_pressure(Body,Body)
#energy_conservation(Body,Body)
energy_conservation_efficient_pressure(Body,Body)
JWL_EOS(Body,371.25e9,3.231e9,4.15,0.95,0.3,1630)
update_smoothing_length(Body,1.2*2.5e-3,1630)
detonation_time_evolve(Body,0,0,6930,1e-7,0)

In [ ]:
momentum_conservation_pressure.parallel_diagnostics(level=4)

 
 Parallel Accelerator Optimizing:  Function momentum_conservation_pressure, 
<ipython-input-7-abb2e019ceab> (1)  


Parallel loop listing for  Function momentum_conservation_pressure, <ipython-input-7-abb2e019ceab> (1) 
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|loop #ID
@jit(nopython=True,fastmath=True,parallel=True)                                                                                                                                                                                                                          | 
def momentum_conservation_pressure(domain,source):                                                                                                                                                                                         

RuntimeError: ignored